## Object Interaction (Object Present Yes or No)

In [2]:
import sqlite3
import pandas as pd

In [ ]:
query = """
WITH RandomSubjects AS (
    -- Select 5 random subjects
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 10
),
RandomFrames AS (
    -- Select 500 random frames per subject
    SELECT DISTINCT d.frame_number, d.video_id
    FROM Detections d
    JOIN RandomSubjects rs ON d.video_id = rs.video_id
    GROUP BY d.frame_number, d.video_id
    ORDER BY RANDOM()
    LIMIT 500
),
SocialContext AS (
    SELECT 
        d.frame_number,
        d.video_id,
        CASE
            WHEN MAX(CASE WHEN d.object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            AND MAX(CASE WHEN d.object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'child and adult present'
            WHEN MAX(CASE WHEN d.object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            THEN 'adult present'
            WHEN MAX(CASE WHEN d.object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'child present'
            ELSE 'alone'
        END as social
    FROM Detections d
    JOIN RandomFrames rf ON d.frame_number = rf.frame_number AND d.video_id = rf.video_id
    GROUP BY d.frame_number, d.video_id
),
ObjectTypes AS (
    -- Create all possible object types
    SELECT 
        'book' as object_type, 5 as object_class UNION ALL
        SELECT 'toy', 6 UNION ALL
        SELECT 'kitchenware', 7 UNION ALL
        SELECT 'screen', 8 UNION ALL
        SELECT 'food', 9 UNION ALL
        SELECT 'other_object', 10
),
FrameObjects AS (
    -- Get unique object presence per frame and object type
    SELECT DISTINCT
        rf.frame_number,
        rf.video_id,
        ot.object_type,
        ot.object_class,
        MAX(CASE WHEN d.object_class IS NOT NULL THEN 1 ELSE 0 END) as object_present
    FROM RandomFrames rf
    CROSS JOIN ObjectTypes ot
    LEFT JOIN Detections d ON 
        rf.frame_number = d.frame_number 
        AND rf.video_id = d.video_id 
        AND ot.object_class = d.object_class
    GROUP BY rf.frame_number, rf.video_id, ot.object_type, ot.object_class
)

SELECT DISTINCT
    fo.video_id,
    fo.frame_number as frame_id,
    rs.child_id as ID,
    fo.object_present,
    fo.object_type,
    rs.age_at_recording as age,
    COALESCE(sc.social, 'alone') as social
FROM 
    FrameObjects fo
    JOIN RandomSubjects rs ON fo.video_id = rs.video_id
    LEFT JOIN SocialContext sc ON fo.frame_number = sc.frame_number AND fo.video_id = sc.video_id
ORDER BY 
    rs.child_id, fo.video_id, fo.frame_number, fo.object_type;
"""

# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    object_df = pd.read_sql_query(query, conn)

# Convert categorical variables to factors
object_df['social'] = pd.Categorical(object_df['social'])
object_df['object_type'] = pd.Categorical(object_df['object_type'])
object_df['ID'] = pd.Categorical(object_df['ID'])

print("Data shape:", object_df.shape)
print("\nNumber of unique subjects:", object_df['ID'].nunique())
print("\nVariable types:")
print(object_df.dtypes)
print("\nSample data (showing first 12 rows to see multiple object types per frame):")
display(object_df.head(12))

Data shape: (3000, 7)

Number of unique subjects: 10

Variable types:
video_id             int64
frame_id             int64
ID                category
object_present       int64
object_type       category
age                float64
social            category
dtype: object

Sample data (showing first 12 rows to see multiple object types per frame):


,video_id,frame_id,ID,object_present,object_type,age,social
0,151,330,257108,0,book,4.25,adult present
1,151,330,257108,0,food,4.25,adult present
2,151,330,257108,0,kitchenware,4.25,adult present
3,151,330,257108,0,other_object,4.25,adult present
4,151,330,257108,0,screen,4.25,adult present
5,151,330,257108,0,toy,4.25,adult present
6,151,1920,257108,0,book,4.25,child present
7,151,1920,257108,0,food,4.25,child present
8,151,1920,257108,0,kitchenware,4.25,child present
9,151,1920,257108,0,other_object,4.25,child present


In [5]:
# Convert columns to categorical type
object_df['object_type'] = object_df['object_type'].astype('category')
object_df['social_context'] = object_df['social'].astype('category')

In [7]:
# save df to csv
object_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/object_test_results.csv', index=False)

# Social Context (Alone Yes or No)

In [21]:
query = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 10
),
AllFrames AS (
    SELECT DISTINCT frame_number, video_id
    FROM Detections
    WHERE video_id IN (SELECT video_id FROM RandomSubjects)
),
RandomFrames AS (
    SELECT frame_number, video_id
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 250
),
FaceInfo AS (
    SELECT 
        rf.video_id,
        rf.frame_number,
        rs.child_id,
        rs.age_at_recording,
        
        -- Binary flags
        CASE WHEN d.object_class = 0 THEN 1 ELSE 0 END AS child_person,
        CASE WHEN d.object_class = 1 THEN 1 ELSE 0 END AS adult_person,
        
        CASE WHEN d.object_class = 2 THEN 1 ELSE 0 END AS child_face,
        CASE WHEN d.object_class = 3 THEN 1 ELSE 0 END AS adult_face,
        
        -- Gaze
        CASE WHEN d.object_class = 2 THEN d.gaze_direction ELSE NULL END AS child_gaze,
        CASE WHEN d.object_class = 3 THEN d.gaze_direction ELSE NULL END AS adult_gaze,

        -- Proximity
        CASE WHEN d.object_class = 2 THEN d.proximity ELSE NULL END AS child_proximity,
        CASE WHEN d.object_class = 3 THEN d.proximity ELSE NULL END AS adult_proximity
    

    FROM RandomFrames rf
    JOIN RandomSubjects rs ON rf.video_id = rs.video_id
    LEFT JOIN Detections d 
        ON d.video_id = rf.video_id 
        AND d.frame_number = rf.frame_number 
        AND d.object_class IN (2, 3)  -- only faces

    GROUP BY rf.video_id, rf.frame_number, rs.child_id, rs.age_at_recording
)

SELECT 
    video_id,
    frame_number AS frame_id,
    child_id AS ID,
    age_at_recording AS age,
    child_person,
    adult_person,
    child_face,
    adult_face,
    child_gaze,
    adult_gaze,
    child_proximity,
    adult_proximity
FROM FaceInfo
ORDER BY child_id, video_id, frame_number;
"""

# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    social_df = pd.read_sql_query(query, conn)

print("Data shape:", social_df.shape)
print("\nNumber of unique subjects:", social_df['ID'].nunique())
print("\nVariable types:")
print(social_df.dtypes)
print("\nSample data (showing first 12 rows to see multiple object types per frame):")
display(social_df.head(12))

Data shape: (250, 12)

Number of unique subjects: 10

Variable types:
video_id             int64
frame_id             int64
ID                   int64
age                float64
child_person         int64
adult_person         int64
child_face           int64
adult_face           int64
child_gaze         float64
adult_gaze         float64
child_proximity    float64
adult_proximity    float64
dtype: object

Sample data (showing first 12 rows to see multiple object types per frame):


,video_id,frame_id,ID,age,child_person,adult_person,child_face,adult_face,child_gaze,adult_gaze,child_proximity,adult_proximity
0,173,2150,257578,4.04,0,0,0,1,NaN,0.0,NaN,0.658033
1,173,13170,257578,4.04,0,0,1,0,1.0,NaN,0.566699,NaN
2,173,13540,257578,4.04,0,0,1,0,1.0,NaN,0.570673,NaN
3,173,14080,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
4,173,18730,257578,4.04,0,0,1,0,1.0,NaN,0.597570,NaN
5,173,19070,257578,4.04,0,0,1,0,1.0,NaN,0.621938,NaN
6,173,21910,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
7,173,22380,257578,4.04,0,0,0,1,NaN,1.0,NaN,0.417363
8,173,22950,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
9,173,28540,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN


In [ ]:
# create column person_present if at least one of adult or child is present
social_df['person_present'] = social_df['child_person'] | social_df['adult_person'] | social_df['child_face'] | social_df['adult_face']
social_df['child_present'] = social_df['child_person'] | social_df['child_face']
social_df['adult_present'] = social_df['adult_person'] | social_df['adult_face']

In [24]:
# save df to csv
social_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/social_test_results.csv', index=False)

In [33]:
import pandas as pd
import re
from datetime import datetime, time

# Load Excel file and prepare data
df = pd.read_excel("/home/nele_pauline_suffo/ProcessedData/quantex_data_sheet.xlsx") 

# remove rows with filename starting with "quantex_at_home_pakistan"
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_pakistan")]
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_id_yyyy")]

# Normalize annotation column
df['is_annotated'] = df['Annotated'].isin(['Yes', 'review', 'in progress'])

# Convert duration to minutes
def time_to_minutes(t):
    if isinstance(t, time):
        return t.hour * 60 + t.minute + (t.second / 60.0)
    elif isinstance(t, str):
        # Parse string format if needed
        try:
            t = datetime.strptime(t, '%H:%M:%S').time()
            return t.hour * 60 + t.minute + (t.second / 60.0)
        except:
            return 0
    return 0

# Add duration in minutes column
df['duration_minutes'] = df['DURATION'].apply(time_to_minutes)

# Get training set (already annotated videos)
train_df = df[df['is_annotated']].copy()
train_length = train_df['duration_minutes'].sum()

# Calculate target duration for val and test (10% each of final dataset)
# train_length is 80%, so divide by 0.8 to get total length, then take 10%
final_total_length = train_length / 0.8
target_length = final_total_length * 0.1

# Get candidate videos for val/test (not yet annotated)
candidates = df[~df['is_annotated']].copy()
candidates = candidates.sort_values('duration_minutes')

# Function to select videos closest to target duration
def select_videos_for_split(candidates_df: pd.DataFrame, target_duration: float, max_videos: int) -> pd.DataFrame:
    """Select videos aiming for target duration with unique IDs."""
    candidates_sorted = candidates_df.copy()  # Create a copy to avoid warnings
    selected = []
    current_duration = 0
    used_ids = set()
    
    while current_duration < target_duration and len(selected) < max_videos:
        # Get remaining candidates using loc
        mask = ~candidates_sorted['ID'].isin(used_ids)
        remaining = candidates_sorted.loc[mask].copy()
        
        if remaining.empty:
            break
            
        # Calculate gap to target using loc
        remaining.loc[:, 'gap_to_target'] = (
            target_duration - (current_duration + remaining['duration_minutes'])
        ).abs()
        
        best_match = remaining.nsmallest(1, 'gap_to_target').iloc[0]
        
        if current_duration + best_match['duration_minutes'] > target_duration * 1.1:
            break
            
        selected.append(best_match)
        used_ids.add(best_match['ID'])
        current_duration += best_match['duration_minutes']
        
        if len(selected) >= 10 and current_duration >= target_duration * 0.9:
            break
    
    return pd.DataFrame(selected)

# Get validation and test sets
candidates_mask = (candidates['duration_minutes'] < target_length * 0.3)  # Reduce max duration threshold
remaining_candidates = candidates.loc[candidates_mask].copy()

# Select validation set
val_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)

# Select test set from remaining videos
test_mask = (~candidates['ID'].isin(val_df['ID']) & 
            (candidates['duration_minutes'] < target_length * 0.3))
remaining_candidates = candidates.loc[test_mask].copy()
test_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)


# Verify unique IDs
train_ids = set(train_df['ID'])
val_ids = set(val_df['ID'])
test_ids = set(test_df['ID'])
# check for overlap in all three sets
overlap = train_ids.intersection(val_ids).union(train_ids.intersection(test_ids)).union(val_ids.intersection(test_ids))

if overlap:
    print("Warning: Found overlapping IDs between validation and test sets!")
    print(f"Overlapping IDs: {overlap}")
    
val_length = val_df['duration_minutes'].sum()
test_length = test_df['duration_minutes'].sum()
total_length = train_length + val_length + test_length

print("✅ Training set (already annotated):")
print(f"Number of videos: {len(train_df)}")

print("\n✅ Validation candidates:")
print(f"Number of videos: {len(val_df)}")
print(val_df[['file name  (generated automatically)', 'duration_minutes']])

print("\n✅ Test candidates:")
print(f"Number of videos: {len(test_df)}")
print(test_df[['file name  (generated automatically)', 'duration_minutes']])

print(f"\nDuration Summary:")
print(f"Training:    {train_length:.2f} minutes ({(train_length/total_length)*100:.1f}%)")
print(f"Validation:  {val_length:.2f} minutes ({(val_length/total_length)*100:.1f}%)")
print(f"Test:        {test_length:.2f} minutes ({(test_length/total_length)*100:.1f}%)")
print(f"Total:       {total_length:.2f} minutes")

print(f"\nTarget duration for val/test: {target_length:.2f} minutes")

Overlapping IDs: {258704.0}
✅ Training set (already annotated):
Number of videos: 75

✅ Validation candidates:
Number of videos: 8
       file name  (generated automatically)  duration_minutes
533  quantex_at_home_id266050_2024_09_15_01              30.0
588  quantex_at_home_id267079_2025_02_11_02              30.0
581  quantex_at_home_id266971_2024_09_06_02              30.0
159  quantex_at_home_id258704_2022_05_10_01              30.0
573  quantex_at_home_id266822_2022_11_14_01              30.0
141  quantex_at_home_id258541_2023_03_26_01              30.0
547  quantex_at_home_id266063_2024_09_19_01              30.0
340  quantex_at_home_id263194_2025_02_15_01               4.9

✅ Test candidates:
Number of videos: 8
       file name  (generated automatically)  duration_minutes
550  quantex_at_home_id266151_2024_09_14_01              30.0
554  quantex_at_home_id266352_2025_01_28_01              30.0
135  quantex_at_home_id258309_2023_03_12_01              30.0
564  quantex_at_home_id

In [ ]:
import pandas as pd
import re
from datetime import datetime, time

# Load Excel file and prepare data
df = pd.read_excel("/home/nele_pauline_suffo/ProcessedData/quantex_data_sheet.xlsx") 

# remove rows with filename starting with "quantex_at_home_pakistan"
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_pakistan")]
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_id_yyyy")]

# Normalize annotation column
df['is_annotated'] = df['Annotated'].isin(['Yes', 'review', 'in progress'])

# Convert duration to minutes
def time_to_minutes(t):
    if isinstance(t, time):
        return t.hour * 60 + t.minute + (t.second / 60.0)
    elif isinstance(t, str):
        # Parse string format if needed
        try:
            t = datetime.strptime(t, '%H:%M:%S').time()
            return t.hour * 60 + t.minute + (t.second / 60.0)
        except:
            return 0
    return 0

# Add duration in minutes column
df['duration_minutes'] = df['DURATION'].apply(time_to_minutes)

# Get training set (already annotated videos)
train_df = df[df['is_annotated']].copy()
train_length = train_df['duration_minutes'].sum()

# Calculate target duration for val and test (10% each of final dataset)
# train_length is 80%, so divide by 0.8 to get total length, then take 10%
final_total_length = train_length / 0.8
target_length = final_total_length * 0.1

# Get candidate videos for val/test (not yet annotated)
candidates = df[~df['is_annotated']].copy()
candidates = candidates.sort_values('duration_minutes')

# Function to select videos closest to target duration
def select_videos_for_split(candidates_df: pd.DataFrame, target_duration: float, max_videos: int) -> pd.DataFrame:
    """Select videos aiming for target duration with unique IDs."""
    candidates_sorted = candidates_df.copy()  # Create a copy to avoid warnings
    selected = []
    current_duration = 0
    used_ids = set()
    
    while current_duration < target_duration and len(selected) < max_videos:
        # Get remaining candidates using loc
        mask = ~candidates_sorted['ID'].isin(used_ids)
        remaining = candidates_sorted.loc[mask].copy()
        
        if remaining.empty:
            break
            
        # Calculate gap to target using loc
        remaining.loc[:, 'gap_to_target'] = (
            target_duration - (current_duration + remaining['duration_minutes'])
        ).abs()
        
        best_match = remaining.nsmallest(1, 'gap_to_target').iloc[0]
        
        if current_duration + best_match['duration_minutes'] > target_duration * 1.1:
            break
            
        selected.append(best_match)
        used_ids.add(best_match['ID'])
        current_duration += best_match['duration_minutes']
        
        if len(selected) >= 10 and current_duration >= target_duration * 0.9:
            break
    
    return pd.DataFrame(selected)

# Get validation and test sets
candidates_mask = (candidates['duration_minutes'] < target_length * 0.3)  # Reduce max duration threshold
remaining_candidates = candidates.loc[candidates_mask].copy()

# Select validation set
val_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)

# Select test set from remaining videos
test_mask = (~candidates['ID'].isin(val_df['ID']) & 
            (candidates['duration_minutes'] < target_length * 0.3))
remaining_candidates = candidates.loc[test_mask].copy()
test_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)


# Verify unique IDs
train_ids = set(train_df['ID'])
val_ids = set(val_df['ID'])
test_ids = set(test_df['ID'])
# check for overlap in all three sets
overlap = train_ids.intersection(val_ids).union(train_ids.intersection(test_ids)).union(val_ids.intersection(test_ids))

if overlap:
    print("Warning: Found overlapping IDs between validation and test sets!")
    print(f"Overlapping IDs: {overlap}")
    
val_length = val_df['duration_minutes'].sum()
test_length = test_df['duration_minutes'].sum()
total_length = train_length + val_length + test_length

print("✅ Training set (already annotated):")
print(f"Number of videos: {len(train_df)}")

print("\n✅ Validation candidates:")
print(f"Number of videos: {len(val_df)}")
print(val_df[['file name  (generated automatically)', 'duration_minutes']])

print("\n✅ Test candidates:")
print(f"Number of videos: {len(test_df)}")
print(test_df[['file name  (generated automatically)', 'duration_minutes']])

print(f"\nDuration Summary:")
print(f"Training:    {train_length:.2f} minutes ({(train_length/total_length)*100:.1f}%)")
print(f"Validation:  {val_length:.2f} minutes ({(val_length/total_length)*100:.1f}%)")
print(f"Test:        {test_length:.2f} minutes ({(test_length/total_length)*100:.1f}%)")
print(f"Total:       {total_length:.2f} minutes")

print(f"\nTarget duration for val/test: {target_length:.2f} minutes")

Overlapping IDs: {258704.0}
✅ Training set (already annotated):
Number of videos: 75

✅ Validation candidates:
Number of videos: 10
       file name  (generated automatically)  duration_minutes
710  quantex_at_home_id284216_2025_03_14_05         30.000000
446  quantex_at_home_id264585_2023_08_16_02         30.000000
436  quantex_at_home_id264514_2025_01_13_02         30.000000
197  quantex_at_home_id260478_2022_11_05_01         30.000000
440  quantex_at_home_id264556_2024_12_18_01         30.000000
191  quantex_at_home_id260178_2023_08_12_02         30.000000
407  quantex_at_home_id264089_2023_05_14_03         30.000000
340  quantex_at_home_id263194_2025_02_15_01          4.900000
107  quantex_at_home_id258239_2020_08_23_03          0.750000
459  quantex_at_home_id264666_2025_02_28_02          1.066667

✅ Test candidates:
Number of videos: 10
       file name  (generated automatically)  duration_minutes
589  quantex_at_home_id267079_2025_02_11_03         30.000000
581  quantex_at_home_